In [1]:
import gzip
from pathlib import Path
import pandas as pd
import numpy as np

print("\nStep 1 ▶ Loading Galaxy Zoo 2 data …")
gz2_path = Path("../data/zoo2MainSpecz.csv.gz")
assert gz2_path.exists(), "File zoo2MainSpecz.csv.gz not found!"

with gzip.open(gz2_path, "rt") as f:
    gz2 = pd.read_csv(
        f,
        usecols=lambda c: c == "specobjid" or c.endswith("_debiased"),
        dtype={"specobjid": str},
        low_memory=False,
    )

print(f"Loaded shape: {gz2.shape}")


Step 1 ▶ Loading Galaxy Zoo 2 data …
Loaded shape: (243500, 38)


In [2]:
"""
Strictly follow the GZ2 decision tree:
Convert debiased probabilities → expected value (EV) with gating,
and output one continuous feature per level.
"""

# ---------------------------------------------------------------------
# Functions
# ---------------------------------------------------------------------
def ev_from_probs(df: pd.DataFrame, mapping: dict) -> pd.Series:
    """
    Compute an ordered-encoding expected value (EV) from a set of debiased
    probability columns:

        EV = sum(code_k * p_k) / sum(p_k)

    Behavior:
    - If the denominator equals 0, return 0 (to avoid NaN).
    - Columns missing from the DataFrame are silently ignored.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing probability columns.
    mapping : dict
        Mapping of {column_name: ordered_code}.

    Returns
    -------
    pd.Series
        Expected value per row.
    """
    idx = df.index
    if not mapping:
        return pd.Series(0.0, index=idx, dtype=float)
    cols = [c for c in mapping if c in df.columns]
    if not cols:
        return pd.Series(0.0, index=idx, dtype=float)
    num = pd.Series(0.0, index=idx, dtype=float)
    den = pd.Series(0.0, index=idx, dtype=float)
    for col, code in mapping.items():
        if col in df.columns:
            p = df[col].fillna(0.0).astype(float)
            num = num.add(p * float(code), fill_value=0.0)
            den = den.add(p, fill_value=0.0)
    ev = num / den.replace(0.0, np.nan)
    return ev.fillna(0.0).astype(float)

def col(df: pd.DataFrame, name: str) -> pd.Series:
    """Safely get a column; if absent, return a zero column with the same index."""
    return df.get(name, pd.Series(0.0, index=df.index, dtype=float)).fillna(0.0).astype(float)


In [3]:
"""
Strictly follow the GZ2 decision tree: use debiased probabilities with gating to
produce expected-value (EV) encodings, yielding one continuous feature per level.
"""

# ---------------------------------------------------------------------
# Unify ID column name
# ---------------------------------------------------------------------
if "specobjid" in gz2.columns and "specObjID" not in gz2.columns:
    gz2 = gz2.rename(columns={"specobjid": "specObjID"})
if "specObjID" not in gz2.columns:
    raise ValueError("未找到 specObjID/specobjid 列。")

# ---------------------------------------------------------------------
# ========= Base probabilities (Stage-1 & main nodes) =========
# ---------------------------------------------------------------------
smooth_prob = col(gz2, "t01_smooth_or_features_a01_smooth_debiased")
disk_prob   = col(gz2, "t01_smooth_or_features_a02_features_or_disk_debiased")
# The star/artifact column is typically excluded from downstream analysis; enable if needed:
# star_artifact_prob = col(gz2, "t01_smooth_or_features_a03_star_or_artifact_debiased")

edgeon_prob = col(gz2, "t02_edgeon_a04_yes_debiased")
not_edgeon_prob = (1.0 - edgeon_prob).clip(0.0, 1.0)

bar_prob_raw    = col(gz2, "t03_bar_a06_bar_debiased")
spiral_prob_raw = col(gz2, "t04_spiral_a08_spiral_debiased")

odd_prob  = col(gz2, "t06_odd_a14_yes_debiased")

# ---------------------------------------------------------------------
# ========= Decision-tree gating =========
# ---------------------------------------------------------------------
# Bar / Spiral questions are asked only on the branch: disk AND not edge-on
bar_gate    = (disk_prob * not_edgeon_prob)
spiral_gate = (disk_prob * not_edgeon_prob)

# Arms subtasks should come after the Bar/Spiral branch; strictly require spiral=Yes:
arms_gate = (spiral_gate * spiral_prob_raw)

# Roundness only on smooth branch
round_gate = smooth_prob
# Edge-on bulge shape only on disk × edge-on
edgeon_gate = (disk_prob * edgeon_prob)

# ---------------------------------------------------------------------
# ========= T07 Roundness (smooth only) =========
# ---------------------------------------------------------------------
round_map = {
    "t07_rounded_a16_completely_round_debiased": 1,
    "t07_rounded_a17_in_between_debiased":      2,
    "t07_rounded_a18_cigar_shaped_debiased":    3,
}
round_ev_raw = ev_from_probs(gz2, round_map)
round_ev = (round_ev_raw * round_gate).astype(float)

# Elliptical probability (quick definition: smooth × completely round)
complete_round = col(gz2, "t07_rounded_a16_completely_round_debiased")
elliptical_prob = (smooth_prob * complete_round).astype(float)

# ---------------------------------------------------------------------
# ========= T03 Bar (disk × not edge-on) =========
# ---------------------------------------------------------------------
bar_prob = (bar_prob_raw * bar_gate).astype(float)

# ---------------------------------------------------------------------
# ========= T04 Spiral (keep raw probability + "effective" probability) =========
# ---------------------------------------------------------------------
spiral_prob = spiral_prob_raw  # keep raw (sometimes useful for interpretation)
spiral_prob_eff = (spiral_prob_raw * spiral_gate).astype(float)  # strictly within disk×not-edge-on branch

# ---------------------------------------------------------------------
# ========= T10 Arm winding (spiral = Yes only) =========
# ---------------------------------------------------------------------
arms_wind_map = {
    "t10_arms_winding_a28_tight_debiased":  1,
    "t10_arms_winding_a29_medium_debiased": 2,
    "t10_arms_winding_a30_loose_debiased":  3,
}
arms_wind_ev_raw = ev_from_probs(gz2, arms_wind_map)
arms_wind_ev = (arms_wind_ev_raw * arms_gate).astype(float)

# ---------------------------------------------------------------------
# ========= T11 Number of arms (spiral = Yes only) =========
# ---------------------------------------------------------------------
arms_num_map = {
    "t11_arms_number_a31_1_debiased":        1,
    "t11_arms_number_a32_2_debiased":        2,
    "t11_arms_number_a33_3_debiased":        3,
    "t11_arms_number_a34_4_debiased":        4,
    "t11_arms_number_a36_more_than_4_debiased": 5,
}
arms_num_ev_raw = ev_from_probs(gz2, arms_num_map)
arms_num_ev = (arms_num_ev_raw * arms_gate).astype(float)

# # "Can't tell" as an uncertainty term (also only meaningful under spiral)
# arms_cant_tell_prob = (col(gz2, "t11_arms_number_a37_cant_tell_debiased") * arms_gate).astype(float)

# ---------------------------------------------------------------------
# ========= T05 Bulge prominence (disk) =========
# ---------------------------------------------------------------------
bulge_map = {
    "t05_bulge_prominence_a10_no_bulge_debiased":        1,
    "t05_bulge_prominence_a11_just_noticeable_debiased": 2,
    "t05_bulge_prominence_a12_obvious_debiased":         3,
    "t05_bulge_prominence_a13_dominant_debiased":        4,
}
bulge_ev_raw = ev_from_probs(gz2, bulge_map)
bulge_ev = (bulge_ev_raw * disk_prob).astype(float)
# If you prefer bulge to be characterized only in bar-free disks, enable the following (more "purified"):
# bulge_ev = (bulge_ev_raw * disk_prob * (1.0 - bar_prob)).astype(float)

# ---------------------------------------------------------------------
# ========= T06/T08 Odd features (global) =========
# ---------------------------------------------------------------------
odd_type_map = {
    "t08_odd_feature_a19_ring_debiased":        1,
    "t08_odd_feature_a20_lens_or_arc_debiased": 2,
    "t08_odd_feature_a21_disturbed_debiased":   3,
    "t08_odd_feature_a22_irregular_debiased":   4,
    "t08_odd_feature_a23_other_debiased":       5,
    "t08_odd_feature_a24_merger_debiased":      6,
    "t08_odd_feature_a38_dust_lane_debiased":   7,
}
odd_type_ev_raw = ev_from_probs(gz2, odd_type_map)
# Nominal aggregation is mainly for dimensionality reduction; interpretability is weak—gate by odd_prob
odd_type_ev = (odd_type_ev_raw * odd_prob).astype(float)

# ---------------------------------------------------------------------
# ========= T09 Bulge shape in edge-on disks (disk × edge-on) =========
# ---------------------------------------------------------------------
bulge_shape_edgeon_map = {
    "t09_bulge_shape_a25_rounded_debiased": 1,
    "t09_bulge_shape_a26_boxy_debiased":    2,
    "t09_bulge_shape_a27_no_bulge_debiased":3,
}
bulge_shape_edgeon_ev_raw = ev_from_probs(gz2, bulge_shape_edgeon_map)
bulge_shape_edgeon_ev = (bulge_shape_edgeon_ev_raw * edgeon_gate).astype(float)

# ---------------------------------------------------------------------
# ========= Consolidated output =========
# ---------------------------------------------------------------------
out = pd.DataFrame({
    "specObjID": gz2["specObjID"],

    # Stage-1: main branches
    "smooth_prob": smooth_prob,
    "disk_prob":   disk_prob,

    # Stage-2 (smooth branch)
    "round_ev": round_ev,
    "elliptical_prob": elliptical_prob,

    # Stage-2 (disk branch)
    "edgeon_prob": edgeon_prob,              # probability of being edge-on for disks
    "bar_prob":    bar_prob,                 # valid only under disk×not-edge-on
    "spiral_prob": spiral_prob,              # raw spiral probability
    "spiral_prob_eff": spiral_prob_eff,      # effective spiral probability under disk×not-edge-on
    "bulge_ev":  bulge_ev,
    "odd_prob":  odd_prob,
    "odd_type_ev": odd_type_ev,              # (optional)

    # Stage-3 (spiral sub-branch)
    "arms_wind_ev": arms_wind_ev,            # winding: valid only when spiral
    "arms_num_ev":  arms_num_ev,             # arm count: valid only when spiral
    # "arms_cant_tell_prob": arms_cant_tell_prob,  # uncertainty (optional)

    # Stage-3 (edge-on sub-branch)
    "bulge_shape_edgeon_ev": bulge_shape_edgeon_ev,
})

# ---------------------------------------------------------------------
# Safety: no NaNs
# ---------------------------------------------------------------------
out = out.fillna(0.0)


In [4]:
out["specObjID"] = out["specObjID"].astype("int64")

In [5]:
selected_columns = [
    "specObjID",
    "bar_prob",
    "bulge_ev",
    "round_ev",
    "arms_num_ev",
    "arms_wind_ev",
    "elliptical_prob",
    "spiral_prob_eff",
    "odd_prob"]

In [6]:
# Save the selected features to a CSV file
output_path = Path("../data/features_gz2.csv")
out[selected_columns].to_csv(output_path, index=False)
print(f"Selected main_morph features saved to {output_path}")

Selected main_morph features saved to ../data/features_gz2.csv


In [7]:
out[selected_columns]

,specObjID,bar_prob,bulge_ev,round_ev,arms_num_ev,arms_wind_ev,elliptical_prob,spiral_prob_eff,odd_prob
0,1802674929645152256,0.556725,2.808975,0.000000,3.186820,1.577940,0.000000,0.928200,0.287000
1,1992983900678285312,0.070713,2.037000,0.345000,0.047095,0.023547,0.000000,0.023547,0.209000
2,1489568922213574656,0.115157,2.363493,0.000000,4.097523,1.263479,0.000000,0.920903,0.532818
3,2924083625089591296,0.466342,1.804130,0.378250,0.707794,0.532703,0.022250,0.298266,0.659000
4,1387165355897546752,0.016973,1.746831,1.325098,0.000000,0.016973,0.000000,0.016973,0.329332
...,...,...,...,...,...,...,...,...,...
243495,2013144842523142144,0.346217,2.532679,0.005329,2.030792,2.789549,0.000000,0.970278,0.227464
243496,333302682783606784,0.000000,1.869204,0.494937,0.000000,0.000000,0.403169,0.000000,0.085198
243497,1959146978059249664,0.000000,0.526005,0.611784,0.211232,0.211232,0.124637,0.105616,0.830432
243498,467329305811118080,0.000000,0.983724,0.727574,0.000000,0.000000,0.571895,0.000000,0.131836
